In [787]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *
from _rulesbuilding import *

In [788]:
rules = pd.read_csv(config.inputs['rules']['fullpath'])
rules = rules[(rules['text_match'].notna()) & (rules['text_exclude'].notna() == False)]

builder = RulesRegexBuilder(GroupedRegexConcatenation('|'))

builder.appendDecorator(RegexDecorator('M', '(', ')'))
builder.appendDecorator(RegexDecorator('S', '^(', ').*$'))
builder.appendDecorator(RegexDecorator('A', '^.*(', ').*$'))

regexes_dict = builder.build(rules) 

In [789]:
signals = {'^.*(' + ('|'.join(v)) + ').*$' : k for k, v in config.inputs['signals'].items()}

In [790]:
data = pd.read_csv(config.inputs['data']['fullpath'])

In [791]:
data = data[data['status'] != 'D']

In [792]:
data = data.sort_values('last_updated').drop_duplicates('item_id',keep='last')

In [793]:
data['service_id'] = data[['description']]
data['service_id'] = data[['service_id']].replace({'service_id':regexes_dict}, regex=True)

In [794]:
data.drop(data[data['service_id'].isin([1, 3, 8, 12, 39]) == False].index, inplace=True)

In [795]:
data['service_name'] = data['service_id'].map(rules.set_index('service_id')['service_name'].to_dict())

In [796]:
data['signal_type'] = data[['description']]
data['signal_type'] = data[['signal_type']].replace({'signal_type': signals}, regex = True)

In [797]:
save_file(config.outputs['local']['fullpath'], data.to_csv())

In [798]:
from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket(config.outputs['cloud']['bucket_name'])
blob = bucket.blob(config.outputs['local']['filename'])
blob.upload_from_filename(config.outputs['local']['fullpath'])

output(blob.public_url)

'https://storage.googleapis.com/antenna-task/processed_data.csv'


In [799]:
# print(data.to_string())